In [1]:
import win32com.client
import pandas as pd
from tqdm import tqdm
import time
import json
import numpy as np
import ast

In [31]:
# 상따 목록 불러오기
PATH = 'c:\\Users\\passi\\Desktop\\programming\\stair_to_heaven\\'


df2 = pd.read_csv(PATH + "catch_highest/data/extracted_data/20201027_date_company_list.csv", usecols=['내일날짜', '상한가'])

for i in range(df2.shape[0]):
    df2['상한가'][i] = ast.literal_eval(df2['상한가'][i])

df2 = df2[['내일날짜', '상한가']]
df2.columns = ['상한가다음날', '상한가종목']

stock_list_name = df2['상한가종목'].tolist()
stock_list_date = df2['상한가다음날'].tolist()

In [32]:
df2

,상한가다음날,상한가종목
0,20200605,"[제이씨현시스템, 피씨디렉트, 이녹스, 삼성중공우, 신성통상, 네온테크]"
1,20200608,"[금호산업우, 이지홀딩스, 일양약품우, 대웅제약, 삼성중공우, 동화약품, 대웅]"
2,20200609,"[유한양행우, 이지홀딩스, 일양약품우, 금강공업우, 일양약품, 한성기업, 삼성중공우..."
3,20200610,"[팅크웨어, 젬백스링크, 한화우, 일양약품우, KG동부제철우, SK네트웍스우, 제일..."
4,20200611,"[한화우, 일양약품우, KG동부제철우, 아이엘사이언스, 에이비엘바이오, 한화솔루션우..."
...,...,...
94,20201022,"[한화우, 대한과학, 한양증권우, KPX생명과학, 수산중공업, 일정실업, 크라운제과..."
95,20201023,"[한양증권우, 대덕1우, 우리기술투자, 에이티넘인베스트, 사조씨푸드]"
96,20201026,"[크라운해태홀딩스우, 에이엔피, 맥스로텍, 대덕1우, 삼아알미늄, 서연탑메탈, 휴림로봇]"
97,20201027,"[삼성물산우B, 코디엠, 휘닉스소재, 호텔신라우, 제일테크노스, 아남전자, 휴림로봇]"


In [4]:
g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
objRq = win32com.client.Dispatch("CpSysDib.CssStgFind")
objRq.SetInputValue(0, 'nyHoIn7oTgSILXpSOTqkuQ')  # 전략 id 요청
objRq.BlockRequest()

cnt = objRq.GetHeaderValue(0)  # 0 - (long) 검색된 결과 종목 수

code = []
name = []

for i in range(cnt):
    code.append(objRq.GetDataValue(0, i))
    name.append(g_objCodeMgr.CodeToName(code[i]))


In [5]:
from datetime import datetime
today = datetime.today().strftime("%Y%m%d")

In [6]:
date_company = pd.DataFrame([[today,code]],columns=['내일날짜','상한가다음날'])

In [9]:
def get_entire_code():
    with open(PATH + "catch_highest/data/extracted_data/kospi_list.json") as kospi_file:
        kospi_data = json.load(kospi_file)

    with open(PATH + "catch_highest/data/extracted_data/kosdaq_list.json") as kosdaq_file:
        kosdaq_data = json.load(kosdaq_file)
    
    kospi_data.update(kosdaq_data)

    return list(kospi_data.keys()), list(kospi_data.values())

In [12]:
date_company.columns = ['상한가다음날', '상한가종목']

stock_list_name = date_company['상한가종목'].tolist()
stock_list_date = date_company['상한가다음날'].tolist()


#분봉 데이터 뽑기
for i in tqdm(range(len(stock_list_date))):
    if int(stock_list_date[i]) > 20201001:
        # try:
        for j in range(len(stock_list_name[i])):
            times = []
            open_price = []
            high_price = []
            low_price = []
            close_price = []
            minute_volume = []
            objStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
            objStockChart.SetInputValue(0, stock_list_name[i][j])  # 종목 코드 입력
            objStockChart.SetInputValue(1, ord('1'))  # Date 설정하여 데이터 받기
            objStockChart.SetInputValue(2, stock_list_date[i])  # todDte 까지
            objStockChart.SetInputValue(3, stock_list_date[i])  # fromDate 까지
            objStockChart.SetInputValue(5, [0, 1, 2, 3, 4, 5, 8])  # 날짜, 시간, 시가,고가,저가,종가,거래량
            objStockChart.SetInputValue(6, ord('m'))  # '차트 주가 - 일간 차트 요청
            objStockChart.SetInputValue(9, ord('0'))  # 무수정주가 사용
            objStockChart.BlockRequest()
            length = objStockChart.GetHeaderValue(3)
            # print(length)
            for k in range(length):
                day = objStockChart.GetDataValue(0, k)
                time = objStockChart.GetDataValue(1, k)
                if time < 1000:  # 0901 출력
                    time = '%04d' % objStockChart.GetDataValue(1, k)
                open = objStockChart.GetDataValue(2, k)
                high = objStockChart.GetDataValue(3, k)
                low = objStockChart.GetDataValue(4, k)
                close = objStockChart.GetDataValue(5, k)
                # volume = objStockChart.GetDataValue(8, k)
                times.append(time)
                open_price.append(open)
                high_price.append(high)
                low_price.append(low)
                close_price.append(close)

            times.reverse()
            open_price.reverse()
            high_price.reverse()
            low_price.reverse()
            close_price.reverse()
            df3 = pd.DataFrame({'time': times, 'open': open_price, 'high': high_price, 'low': low_price,'close': close_price}, columns = ['time','open','high','low','close'])
            df3.to_csv(PATH + "catch_highest/data/minute_stock_data/" + str(stock_list_date[i]) + '_'+ g_objCodeMgr.CodeToName(stock_list_name[i][j]) + '.csv',
                encoding='utf-8-sig')
        # except Exception as e:
        #     print("Error code: " + str(code))
        #     print("Error name: " + str(e))

    else:
        continue

100%|██████████| 1/1 [00:00<00:00,  3.58it/s]


In [63]:
stock_list_name[0]

['A008700', 'A008775', 'A02826K', 'A038010', 'A090710', 'A304840']

In [19]:
#분봉 데이터 파일들을 모두 리스트에 넣기
PATH = 'c:\\Users\\passi\\Desktop\\programming\\stair_to_heaven\\'
file_list = os.listdir(PATH + "catch_highest/data/minute_stock_data/")
file_list.sort()
if '.DS_Store' in file_list:
    file_list = file_list[1:]
else:
    None

In [21]:
file_list.reverse()
file_list = file_list[0:num]
file_list.sort()


100

In [24]:
file_list

['20201005_THE MIDONG.csv',
 '20201005_디케이디앤아이.csv',
 '20201005_라닉스.csv',
 '20201005_새로닉스.csv',
 '20201005_신테카바이오.csv',
 '20201005_한류AI센터.csv',
 '20201006_네오크레마.csv',
 '20201006_맥스로텍.csv',
 '20201006_부산주공.csv',
 '20201006_셀리드.csv',
 '20201006_오르비텍.csv',
 '20201006_인베니아.csv',
 '20201006_인지컨트롤스.csv',
 '20201006_현대공업.csv',
 '20201007_TBH글로벌.csv',
 '20201007_나인테크.csv',
 '20201007_에스와이.csv',
 '20201007_이수화학.csv',
 '20201007_한창산업.csv',
 '20201008_SV인베스트먼트.csv',
 '20201008_까뮤이앤씨.csv',
 '20201008_엠에스씨.csv',
 '20201008_자이글.csv',
 '20201008_한국유니온제약.csv',
 '20201012_골드퍼시픽.csv',
 '20201012_동일철강.csv',
 '20201012_모아텍.csv',
 '20201012_신성이엔지.csv',
 '20201012_쌍용양회우.csv',
 '20201012_팬젠.csv',
 '20201012_한농화성.csv',
 '20201013_남성.csv',
 '20201013_쌍용양회우.csv',
 '20201013_에스에이티.csv',
 '20201013_엑세스바이오.csv',
 '20201013_오성첨단소재.csv',
 '20201014_SDN.csv',
 '20201014_네온테크.csv',
 '20201014_대창솔루션.csv',
 '20201014_모다이노칩.csv',
 '20201014_모아텍.csv',
 '20201014_쌍용양회우.csv',
 '20201014_알에프세미.csv',
 '20201014_우림기계.csv',
 '2